## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config();

## Get the slice and the nodes

In [ ]:
slice_name = '{SLICE_NAME}'
# naming nodes
node1_name = 'scion01'
node2_name = 'scion02'
node3_name = 'scion03'
node4_name = 'scion04'
node5_name = 'scion05'
# creating l2 networks
network_name12='net12'
network_name14='net14'
network_name13='net13'
network_name23='net23'
network_name25='net25'
network_name34='net34'
network_name35='net35'
# namic NICs
node1a_nic_name = 'nic1a'
node1b_nic_name = 'nic1b'
node1c_nic_name = 'nic1c'
node2a_nic_name = 'nic2a'
node2b_nic_name = 'nic2b'
node2c_nic_name = 'nic2c'
node3a_nic_name = 'nic3a'
node3b_nic_name = 'nic3b'
node3c_nic_name = 'nic3c'
node3d_nic_name = 'nic3d'
node4a_nic_name = 'nic4a'
node4b_nic_name = 'nic4b'
node5a_nic_name = 'nic5a'
node5b_nic_name = 'nic5b'

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
##get the nodes
node1 = slice.get_node(name=node1_name)
node2 = slice.get_node(name=node2_name)
node3 = slice.get_node(name=node3_name)
node4 = slice.get_node(name=node4_name)
node5 = slice.get_node(name=node5_name)

nodes = [node1, node2, node3, node4, node5]

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet1 = IPv4Network("{SUBNET1}")
subnet2 = IPv4Network("{SUBNET2}")
subnet3 = IPv4Network("{SUBNET3}")
subnet4 = IPv4Network("{SUBNET4}")
subnet5 = IPv4Network("{SUBNET5}")
subnet6 = IPv4Network("{SUBNET6}")
subnet7 = IPv4Network("{SUBNET7}")

available_ips1 = list(subnet1)[1:]
available_ips2 = list(subnet2)[1:]
available_ips3 = list(subnet3)[1:]
available_ips4 = list(subnet4)[1:]
available_ips5 = list(subnet5)[1:]
available_ips6 = list(subnet6)[1:]
available_ips7 = list(subnet7)[1:]

# OS SETUP

In [ ]:
nodes = [node1, node2, node3, node4, node5]

In [ ]:
nodes = [node1, node2, node3, node4, node5]
for node in nodes:
    result = node.upload_file('{PATH_TO_PRIVATE_SLIVER_KEY_ON_LOCAL_FILEBROWSER}', '{PATH_TO_PRIVATE_SLIVER_KEY_ON_NODE}')
    stdout, stderr = node.execute(f'chmod 600 PATH_TO_PRIVATE_SLIVER_KEY_ON_NODE')

In [ ]:
for node in nodes:
    stdout, stderr = node.execute('sudo mkdir /etc/scion')
    stdout, stderr = node.execute('sudo mkdir -p /var/lib/scion')
    stdout, stderr = node.execute('sudo chown -R ubuntu:ubuntu /etc/scion/')
    stdout, stderr = node.execute('sudo chown -R ubuntu:ubuntu /var/lib/scion/')
    stdout, stderr = node.execute('mkdir -p /etc/scion/certs')
    stdout, stderr = node.execute('mkdir -p /etc/scion/crypto/as')
    stdout, stderr = node.execute('mkdir -p /etc/scion/keys')

## Downloading and Installing software

In [ ]:
for node in nodes:
    result = node.execute('wget https://github.com/scionproto/scion/releases/download/v0.9.1/scion_v0.9.1_amd64_linux.tar.gz')
    stdout, stderr = node.execute('sudo mkdir /usr/local/scion')
    stdout, stderr = node.execute('sudo tar xfz scion_v0.9.1_amd64_linux.tar.gz -C /usr/local/scion')

# AS Configuration

## STEP1 - Topology files

In [ ]:
## Uploade the topologies to the folder "topologies" of the directory where the current file resides
i = 1
for node in nodes:
    result = node.upload_file(f'topologies/topology{i}.json', '/etc/scion/topology.json')
    i += 1

In [ ]:
## replacing IPs in files with the actual IPs used in topology
for node in nodes:
    stdout, stderr = node.execute("sed -i 's/10.0.0.1/{YOUR_NODE1_IP}/g' /etc/scion/topology.json")
    stdout, stderr = node.execute("sed -i 's/10.0.0.2/{YOUR_NODE2_IP}/g' /etc/scion/topology.json")
    stdout, stderr = node.execute("sed -i 's/10.0.0.3/{YOUR_NODE3_IP}/g' /etc/scion/topology.json")
    stdout, stderr = node.execute("sed -i 's/10.0.0.4/{YOUR_NODE4_IP}/g' /etc/scion/topology.json")
    stdout, stderr = node.execute("sed -i 's/10.0.0.5/{YOUR_NODE5_IP}/g' /etc/scion/topology.json")

## Step 2 - Generate the Required Certificates

In [ ]:
## Uploading wide topology file node1
result = node1.upload_file(f'topologies/TutorialDeploymentTopology.topo', 'TutorialDeploymentTopology.topo')

In [ ]:
## Generating the required certificates in node 1
stdout, stderr = node1.execute("/usr/local/scion/scion-pki testcrypto -t TutorialDeploymentTopology.topo")

## Copying certs to nodes
Steps to reproduce:
- SSH into node 1
- NOTE: in the following commands 'i' represents the number of the node. Replace i with 1 up and till 5. If you have another naming convention for your nodes like 'scion01' then use that naming convention
- Execute this line 'scp -i {yourPrivateSilverKeyName} -r gen/ASffaa_1_{i}/crypto ubuntu@Node{i}:/etc/scion'
- Then Execute this 'scp -i KNIT6TutorialSliverKey -r gen/trcs/ISD42-B1-S1.trc ubuntu@Node{i}:/etc/scion/certs/'

source: https://scion--4435.org.readthedocs.build/en/4435/tutorials/deploy.html (under section "Step 2 - Generate the Required Certificates")

In [ ]:
## copying certs to nodes
# fails because needs prompt confirmation for adding the certs in the terminal - not fixed yet.

# for i in range (1,6):
#     print(i)
#     stdout, stderr = node1.execute(f"echo 'yes' | scp -i {PATH_TO_PRIVATE_SLIVER_KEY} -r gen/ASffaa_1_{i}/crypto ubuntu@scion0{i}:/etc/scion")
#     stdout, stderr = node1.execute(f"echo 'yes' | scp -i {PATH_TO_PRIVATE_SLIVER_KEY} -r gen/trcs/ISD42-B1-S1.trc ubuntu@scion0{i}:/etc/scion/certs/")

## Generate Forwarding Secret Keys

In [ ]:
for node in nodes:
    stdout, stderr = node.execute("dd if=/dev/urandom bs=16 count=1 | base64 - > /etc/scion/keys/master0.key")
    stdout, stderr = node.execute("dd if=/dev/urandom bs=16 count=1 | base64 - > /etc/scion/keys/master1.key")

## Service Configuration Files

In [ ]:
## the toml configuration files are in the toml_files folder of working directory
for node in nodes:
    result = node.upload_file(f'toml_files/br.toml', '/etc/scion/br.toml')
    result = node.upload_file(f'toml_files/cs.toml', '/etc/scion/cs.toml')
    result = node.upload_file(f'toml_files/dispatcher.toml', '/etc/scion/dispatcher.toml')
    result = node.upload_file(f'toml_files/sd.toml', '/etc/scion/sd.toml')

## Start the Services

In [ ]:
for node in nodes:
    stdout, stderr = node.execute(f'/usr/local/scion/router --config /etc/scion/br.toml > router.log 2>&1 &')
    stdout, stderr = node.execute(f'/usr/local/scion/dispatcher --config /etc/scion/dispatcher.toml > dispatcher.log 2>&1 &')
    stdout, stderr = node.execute(f'/usr/local/scion/control --config /etc/scion/cs.toml > control.log 2>&1 &')
    stdout, stderr = node.execute(f'/usr/local/scion/daemon --config /etc/scion/sd.toml > daemon.log 2>&1 &')

## Testing the connectivity

In [ ]:
for node in nodes:
    stdout, stderr = node.execute('/usr/local/scion/scion address')

In [ ]:
#ping from node1 to node5
stdout, stderr = node1.execute('/usr/local/scion/scion ping 42-ffaa:1:5,127.0.0.1 -c 5')

In [ ]:
#verifying different path availability
stdout, stderr = node1.execute('/usr/local/scion/scion showpaths 42-ffaa:1:5')